In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1498498/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_49_4,0.949778,-0.252126,-0.337342,0.781369,0.007889,0.743316,0.936836,0.011003,0.035393,0.088817,0.898772,0.089509,139.684694,326.074433,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_49_5,0.949701,-0.259450,-0.341694,0.775570,0.007901,0.747664,0.939884,0.011295,0.035387,0.088885,0.898617,0.089577,139.681635,326.071374,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_49_6,0.949689,-0.260036,-0.336572,0.770005,0.007903,0.748012,0.936297,0.011575,0.035318,0.088897,0.898591,0.089588,139.681129,326.070868,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_49_7,0.949448,-0.275119,-0.355925,0.753352,0.007940,0.756966,0.949854,0.012413,0.035329,0.089108,0.898107,0.089802,139.671605,326.061344,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_3,0.949438,-0.239653,-0.320816,0.787642,0.007942,0.735912,0.925259,0.010687,0.035515,0.089118,0.898085,0.089811,139.671179,326.060919,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_6,0.050394,-0.020107,-0.054390,0.010838,0.149158,0.605579,0.738622,0.175236,0.175968,0.386210,-0.914049,0.389215,133.805500,320.195239,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.033901,-0.028341,-0.047848,0.022286,0.151749,0.610467,0.734039,0.173208,0.168318,0.389549,-0.947293,0.392581,133.771061,320.160800,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.009420,-0.043520,-0.059649,-0.002397,0.155594,0.619478,0.742306,0.177580,0.172106,0.394454,-0.996637,0.397524,133.721013,320.110752,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,-0.008880,-0.051068,-0.070640,-0.019960,0.158468,0.623960,0.750005,0.180692,0.174596,0.398081,-1.033523,0.401179,133.684402,320.074141,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
